In [1]:
from PIL import Image
import numpy as np
import os

In [2]:
def resize(input_dir, output_dir, target_size=(32,32)):
    os.makedirs(output_dir, exist_ok=True)

    for kuva in os.listdir(input_dir):
        img_path = os.path.join(input_dir, kuva)
        img = Image.open(img_path)
        resized = img.resize(target_size, Image.BICUBIC)
        resized = resized.convert('RGB')
        resized.save(os.path.join(output_dir, kuva))

resize("./data/angry", "./data/resized_angry")
resize("./data/happy", "./data/resized_happy")
resize("./data/sad", "./data/resized_sad")

C:\Users\aatus\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\PIL\Image.py:970: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  warnings.warn(


In [16]:
import torch
from torchvision import transforms
import torch.nn.functional as F

angry_path = "./data/resized_angry"
happy_path = "./data/resized_happy"
sad_path = "./data/resized_sad"

valid_extensions = ('.jpg', '.jpeg', '.png')

data = []
labels = []

to_tensor = transforms.ToTensor()

for kuva in os.listdir(angry_path):
    if kuva.endswith(valid_extensions):
        img_path = os.path.join(angry_path, kuva)
        img = Image.open(img_path)
        tensori = to_tensor(img)
        data.append(tensori)
        labels.append(0)

for kuva in os.listdir(happy_path):
    if kuva.endswith(valid_extensions):
        img_path = os.path.join(happy_path, kuva)
        img = Image.open(img_path)
        tensori = to_tensor(img)
        data.append(tensori)
        labels.append(1)

for kuva in os.listdir(sad_path):
    if kuva.endswith(valid_extensions):
        img_path = os.path.join(sad_path, kuva)
        img = Image.open(img_path)
        tensori = to_tensor(img)
        data.append(tensori)
        labels.append(2)

data = torch.stack(data)
labels = torch.tensor(labels)
labels = F.one_hot(labels)

In [17]:
print(data.shape)
print(labels.shape)

torch.Size([288, 3, 32, 32])
torch.Size([288, 3])


In [18]:
from torch.utils.data import DataLoader, TensorDataset, random_split

labeled_data = TensorDataset(data, labels)
train_len = int(0.8*len(labeled_data))
test_len = len(labeled_data) - train_len

train_data, test_data = random_split(labeled_data, [train_len, test_len])


loaders = {
    "train": DataLoader(train_data, 
                        batch_size= 16, 
                        shuffle=True,),
    "test": DataLoader(test_data,
                       batch_size=16,
                       shuffle=True)
}

In [19]:
loaders

{'train': <torch.utils.data.dataloader.DataLoader at 0x168efe900>,
 'test': <torch.utils.data.dataloader.DataLoader at 0x168efc6b0>}

In [20]:
import torch.nn as nn
import torch.optim as optim
import matplotlib.pyplot as plt

In [ ]:
class CNN(nn.Module):

    def __init__(self, x):
        super(CNN, self).__init__()

        self.conv1 = nn.Conv2d(3, 32, kernel_size=5)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=5)
        self.conv3 = nn.Conv2d(64, 128, kernel_size=5)
        self.conv2_dropout = nn.Dropout2d()
        self.conv3_dropout = nn.Dropout2d()
        self.fc1 = 